# KHÁM PHÁ VÀ TIỀN XỬ LÝ DỮ LIỆU

## Import thư viện cần thiết

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Thu thập dữ liệu

### Thông tin chung

- **Chủ đề dữ liệu**: Thông tin về Anime (Phim hoạt hình Nhật Bản)
- **Cách thu thập dữ liệu**: Bộ dữ liệu được thu thập từ nền tảng MyAnimeList, một cộng đồng và cơ sở dữ liệu trực tuyến phổ biến dành cho những người đam mê anime và manga. Nền tảng này cung cấp thông tin về các anime, hồ sơ người dùng và điểm của người dùng đánh giá cho các anime khác nhau. Chúng em sử dụng ngôn ngữ lập trình Python, cùng Selenium để trích xuất dữ liệu từ web.

## Khám phá và tiền xử lý dữ liệu

### Đọc dữ liệu từ file csv

In [2]:
anime_df = pd.read_csv('D:/Intro2DS_FinalProject/Data_collection/animes.csv')

#### Hiển thị 5 hàng đầu tiên trong tập dữ liệu

In [3]:
anime_df.head()

,ID,Title,Episodes,Type,Aired,Producers,Studios,Source,Genres,Duration,Score,Rating count,Rank,Popularity
0,1.0,Cowboy Bebop,26,TV,"Apr 3, 1998 to Apr 24, 1999",Bandai Visual,Sunrise,Original,"Action, Award Winning, Sci-Fi",24 min. per ep.,8.75,988000.0,482,43.0
1,5.0,Cowboy Bebop: Tengoku no Tobira,1,Movie,"Sep 1, 2001","Sunrise, Bandai Visual",Bones,Original,"Action, Sci-Fi",1 hr. 55 min.,8.38,219779.0,2092,631.0
2,6.0,Trigun,26,TV,"Apr 1, 1998 to Sep 30, 1998",Victor Entertainment,Madhouse,Manga,"Action, Adventure, Sci-Fi",24 min. per ep.,8.22,380049.0,3602,258.0
3,7.0,Witch Hunter Robin,26,TV,"Jul 3, 2002 to Dec 25, 2002","Bandai Visual, Dentsu, Victor Entertainment",Sunrise,Original,"Action, Drama, Mystery, Supernatural",25 min. per ep.,7.24,44600.0,30982,1908.0
4,8.0,Bouken Ou Beet,52,TV,"Sep 30, 2004 to Sep 29, 2005","TV Tokyo, Dentsu",Toei Animation,Manga,"Action, Adventure, Fantasy",23 min. per ep.,6.93,6891.0,45862,5485.0


### Số hàng và số cột của tập dữ liệu

In [4]:
nrows, ncols = anime_df.shape
print('Số hàng của dataset:', nrows)
print('Số cột của dataset:', ncols)

Số hàng của dataset: 25012
Số cột của dataset: 14


### Ý nghĩa của mỗi hàng

Mỗi hàng chứa thông tin về 1 bộ anime như tên, thể loại, số tập, thời lượng, nhà sản xuất,... và xếp hạng dựa trên một số tiêu chí cũng như là điểm đánh giá trung bình cho anime đó do người dùng đánh giá trên nền tảng MyAnimeList.

### Loại bỏ những hàng không có dữ liệu

In [5]:
anime_df.dropna(how='all', inplace=True)

### Loại bỏ những hàng trùng lặp

In [6]:
num_duplicates = anime_df.duplicated().sum()
print(f"Số hàng trùng lặp: {num_duplicates}")

Số hàng trùng lặp: 19


In [7]:
anime_df = anime_df.drop_duplicates()

In [8]:
nrows, ncols = anime_df.shape
print('Số hàng của dataset sau khi xử lý:', nrows)

Số hàng của dataset sau khi xử lý: 24990


### Ý nghĩa của mỗi cột

In [9]:
# Xem tên của từng cột
anime_df.columns

Index(['ID', 'Title', 'Episodes', 'Type', 'Aired', 'Producers', 'Studios',
       'Source', 'Genres', 'Duration', 'Score', 'Rating count', 'Rank',
       'Popularity'],
      dtype='object')

#### Ý nghĩa của mỗi cột:
- Cột đầu tiên cho biết ID của anime (ID)
- Cột thứ 2 cho biết tên gốc của anime (Title)
- Cột thứ 3 cho biết số tập của anime (Episodes)
- Cột thứ 4 cho biết anime thuộc dạng phim dài tập hay movie,... (Type)
- Cột thứ 5 cho biết ngày bắt đầu và kết thúc việc phát sóng anime (Aired)
- Cột thứ 6 cho biết công ty hoặc nhà sản xuất của anime (Producers)
- Cột thứ 7 cho biết hãng làm phim hoạt hình làm ra anime (Studios)
- Cột thứ 8 cho biết nguồn tài liệu để làm ra anime (Source)
- Cột thứ 9 cho biết thể loại của anime, một anime có thể thuộc nhiều thể loại được phân tách bởi dấu phẩy (Genres)
- Cột thứ 10 cho biết thời lượng của mỗi tập trong anime (Duration)
- Cột thứ 11 cho biết điểm số trung bình đánh giá cho anime bởi người dùng (Score)
- Cột thứ 12 cho biết số lượng người dùng đã chấm điểm cho anime (Rating count)
- Cột thứ 13 cho biết xếp hạng của anime dựa trên Score, nếu cùng điểm sẽ so sánh các tiêu chí khác (Rank)
- Cột thứ 14 cho biết xếp hạng độ phổ biến của anime (Popularity)

Cột thứ 12 cho biết số lượng người dùng đã chấm điểm cho anime nhưng có tên Rating count chưa phù hợp lắm nên ta sẽ đổi tên cột thành Scored By.

In [10]:
anime_df.rename(columns={'Rating count': 'Scored By'}, inplace=True)

In [11]:
anime_df.head()

,ID,Title,Episodes,Type,Aired,Producers,Studios,Source,Genres,Duration,Score,Scored By,Rank,Popularity
0,1.0,Cowboy Bebop,26,TV,"Apr 3, 1998 to Apr 24, 1999",Bandai Visual,Sunrise,Original,"Action, Award Winning, Sci-Fi",24 min. per ep.,8.75,988000.0,482,43.0
1,5.0,Cowboy Bebop: Tengoku no Tobira,1,Movie,"Sep 1, 2001","Sunrise, Bandai Visual",Bones,Original,"Action, Sci-Fi",1 hr. 55 min.,8.38,219779.0,2092,631.0
2,6.0,Trigun,26,TV,"Apr 1, 1998 to Sep 30, 1998",Victor Entertainment,Madhouse,Manga,"Action, Adventure, Sci-Fi",24 min. per ep.,8.22,380049.0,3602,258.0
3,7.0,Witch Hunter Robin,26,TV,"Jul 3, 2002 to Dec 25, 2002","Bandai Visual, Dentsu, Victor Entertainment",Sunrise,Original,"Action, Drama, Mystery, Supernatural",25 min. per ep.,7.24,44600.0,30982,1908.0
4,8.0,Bouken Ou Beet,52,TV,"Sep 30, 2004 to Sep 29, 2005","TV Tokyo, Dentsu",Toei Animation,Manga,"Action, Adventure, Fantasy",23 min. per ep.,6.93,6891.0,45862,5485.0


### Kiểu dữ liệu hiện tại của mỗi cột là gì? Có cột nào có kiểu dữ liệu không phù hợp không? 


Ta sẽ thử xem xét các kiểu dữ liệu của của từng cột đã phù hợp chưa. 

In [12]:
anime_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24990 entries, 0 to 25011
Data columns (total 14 columns):
ID            24990 non-null float64
Title         24990 non-null object
Episodes      24990 non-null object
Type          24990 non-null object
Aired         24990 non-null object
Producers     24990 non-null object
Studios       24990 non-null object
Source        24990 non-null object
Genres        10879 non-null object
Duration      24990 non-null object
Score         16943 non-null float64
Scored By     16943 non-null float64
Rank          24990 non-null object
Popularity    24990 non-null float64
dtypes: float64(4), object(10)
memory usage: 2.9+ MB


Sau khi xem qua các cột dữ liệu ta thấy các cột nên có dữ liệu dạng số nhưng lại có dữ liệu object là: 
- Episodes
- Rank

Do đó ta sẽ chuẩn hóa kiểu dữ liệu cho các cột này về dạng số.

In [13]:
col_convert = ['Episodes', 'Rank']
for col in col_convert:
    anime_df[col] = pd.to_numeric(anime_df[col], errors='coerce')

In [14]:
anime_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24990 entries, 0 to 25011
Data columns (total 14 columns):
ID            24990 non-null float64
Title         24990 non-null object
Episodes      24725 non-null float64
Type          24990 non-null object
Aired         24990 non-null object
Producers     24990 non-null object
Studios       24990 non-null object
Source        24990 non-null object
Genres        10879 non-null object
Duration      24990 non-null object
Score         16943 non-null float64
Scored By     16943 non-null float64
Rank          19849 non-null float64
Popularity    24990 non-null float64
dtypes: float64(6), object(8)
memory usage: 2.9+ MB


Các kiểu dữ liệu còn lại đã phù hợp với nội dung nên ta sẽ tiếp tục phần tiếp theo

### Với mỗi cột có kiểu dữ liệu dạng số, các giá trị được phân bố như thế nào?

#### Đầu tiên ta sẽ lấy tên các cột có dữ liệu dạng số và lưu vào `numeric_columns`

In [15]:
numeric_columns = anime_df.select_dtypes(include='number').columns[:] 
numeric_columns

Index(['ID', 'Episodes', 'Score', 'Scored By', 'Rank', 'Popularity'], dtype='object')

#### Sau đó, ta sẽ xem tỉ lệ giá trị bị thiếu trong các cột dữ liệu kiểu số là bao nhiêu:

In [16]:
percentage_missing_dic = {}
for col in numeric_columns:
    percentage_missing = anime_df[col].isna().mean() * 100
    percentage_missing_dic[col] = percentage_missing
    print(f'Phần trăm giá trị còn thiếu của cột \'{col}\' là {percentage_missing:.2f}%')

Phần trăm giá trị còn thiếu của cột 'ID' là 0.00%
Phần trăm giá trị còn thiếu của cột 'Episodes' là 1.06%
Phần trăm giá trị còn thiếu của cột 'Score' là 32.20%
Phần trăm giá trị còn thiếu của cột 'Scored By' là 32.20%
Phần trăm giá trị còn thiếu của cột 'Rank' là 20.57%
Phần trăm giá trị còn thiếu của cột 'Popularity' là 0.00%


Sau khi xem qua ta thấy dữ liệu đang bị thiếu nhiều nhất ở những cột quan trọng như Score (điểm của Anime), Scored By (số người đánh giá) với 32.20%. Hai cột này có nhiều giá trị trong quá trình tính toán nên chúng em sẽ chọn giữ lại nó và tiến hành xử lý các giá trị bị thiếu.

#### Xử lý các giá trị bị thiếu ở các cột Score và Scored By

Thử kiểm tra xếp hạng độ phổ biến (Popularity) của những anime thiếu giá trị Score và Scored By.

In [17]:
null_rows = anime_df[anime_df['Score'].isnull()]

# Xếp hạng độ phổ biến của cột 'Popularity' trong các hàng có giá trị null
null_rows[['Popularity']].sort_values(by='Popularity', ascending=True).head(20)

,Popularity
22414,1018.0
20708,1546.0
22581,1707.0
24165,1818.0
21911,1989.0
23525,2677.0
21378,2745.0
20522,2764.0
23052,2789.0
24741,2865.0


Mặc dù tỉ lệ giá trị bị thiếu ở 2 cột Score và Scored By hơn 30% nhưng qua kiếm tra ta thấy những anime bị thiếu dữ liệu này có độ phổ biến không quá cao (từ 1018 trở xuống). Với mục tiêu phân tích xu hướng anime thịnh hành, đồng thời để tránh việc bị lệch phân phối dữ liệu khi điền giá trị bị thiếu bằng trung vị/ mô hình thì nhóm chọn cách xóa những hàng (anime) thiếu dữ liệu này.

In [18]:
anime_df = anime_df.dropna(subset=['Score'])

anime_df.shape

(16943, 14)

#### Tiếp theo ta xem qua mô tả của các cột có dữ liệu dạng số xem có gì bất thường không (tối thiểu, tối đa)

In [19]:
anime_df.describe()

,ID,Episodes,Score,Scored By,Rank,Popularity
count,16943.000000,16864.000000,16943.000000,1.694300e+04,13053.000000,16943.000000
mean,24144.326979,11.266544,6.398052,2.984130e+04,68191.011721,9183.380570
std,17985.647036,49.119867,0.892995,1.182212e+05,39301.829723,5513.639794
min,1.000000,1.000000,1.880000,1.030000e+02,12.000000,1.000000
25%,5680.500000,1.000000,5.780000,3.715000e+02,34152.000000,4473.000000
50%,24629.000000,2.000000,6.380000,1.673000e+03,68212.000000,8985.000000
75%,38861.500000,12.000000,7.040000,1.052350e+04,102212.000000,13584.000000
max,57160.000000,3057.000000,9.320000,2.858563e+06,136132.000000,21347.000000


Ta xem xét từng cột và nhận xét:
- ID có giá trị từ 1 đến 57160 mặc dù số lượng anime là 16943, vậy nên ta có thể suy ra ID của các anime là không liên tục.
- Cột Score có giá trị từ 1.88 đến 9.32 đây là 1 khoảng hợp lý cho đánh giá điểm số của anime, không có anime nào 10 và không có anime bị đánh giá 0 điểm. Điểm trung bình của các anime nằm quanh khoảng 6.4, với phần lớn từ 5 đến 7.
- Số lượng tập có sự chênh lệch lớn giữa các anime.
- Độ phổ biến và số lượng người đánh giá cho thấy có sự chênh lệch đáng kể về mức độ phổ biến giữa các anime.
- Các cột đều có khoảng giá trị phù hợp với ý nghĩa của từng cột.

### Với mỗi cột có kiểu dữ liệu dạng phân loại, các giá trị được phân bố như thế nào?

Ở phần này sẽ có 2 dạng:
- Cột có giá trị đơn: Title, Type, Source.
- Cột có nhiều giá trị: Genres, Producers, Studios

#### Do đó đầu tiên ta sẽ xử lý trên các cột đơn trước

Đầu tiên lưu tên các cột có một giá trị vào `single_columns`

In [20]:
single_columns = ['Title', 'Type', 'Source', 'Duration', 'Aired']

Sau đó tính phần trăm giá trị còn thiếu của mỗi cột này

Ở bộ dữ liệu này các dữ liệu `NULL` đang được đánh dấu bằng nhiều chuỗi string khác nhau như: `Unknown`, `Not available`, `None found, add some`. Do đó ta sẽ thay thế các giá trị trên cũng như cả các dữ liệu `NULL` bằng một giá trị nhất quán `UNKNOWN` để đánh dấu các giá trị bị thiếu. 

In [21]:
values_to_replace = ["Unknown", "Not available", "None found, add some", np.nan]

# Thay thế các giá trị không mong muốn bằng "UNKNOWN"
for column in anime_df.columns:     
    if anime_df[column].dtype == "object":
        anime_df[column] = anime_df[column].replace(values_to_replace, "UNKNOWN")

In [22]:
for col in single_columns:
    percentage_missing = (anime_df[col] == 'UNKNOWN').mean() * 100
    percentage_missing_dic[col] = percentage_missing
    print(f'Phần trăm giá trị còn thiếu của cột \'{col}\' là {percentage_missing:.2f}%')

Phần trăm giá trị còn thiếu của cột 'Title' là 0.00%
Phần trăm giá trị còn thiếu của cột 'Type' là 0.01%
Phần trăm giá trị còn thiếu của cột 'Source' là 6.99%
Phần trăm giá trị còn thiếu của cột 'Duration' là 0.09%
Phần trăm giá trị còn thiếu của cột 'Aired' là 0.06%


Ta thấy các cột này có giá trị thiếu rất ít có thể do các dữ liệu này là các thông tin cơ bản của một anime nên hầu như tất cả các anime được sản xuất đều phải có các thông tin này

Lưu số lượng giá trị khác nhau và đếm số lượng từng loại và lưu vào mảng `single_cate_profile`

In [23]:
# Define single_columns explicitly (assuming it contains the columns to analyze)
single_columns = ["Title", "Type", "Source", "Duration", "Aired"]

# Create an empty DataFrame with the correct structure
single_cate_profile = pd.DataFrame(index=["num_diff_vals", "distribution"], columns=single_columns)

# Calculate the number of unique values
num_diff_vals = anime_df[single_columns].nunique()

# Calculate the distribution
distribution = anime_df[single_columns].apply(lambda column: column.value_counts().to_dict())

# Assign the calculated values to the DataFrame
single_cate_profile.loc["num_diff_vals"] = num_diff_vals
single_cate_profile.loc["distribution"] = distribution

# Display the resulting DataFrame
single_cate_profile


,Title,Type,Source,Duration,Aired
num_diff_vals,16942,10,17,327,12167
distribution,{'Shen Lan Qi Yu Wushuang Zhu: Tianmo Pian': 2...,"{'TV': 4702, 'OVA': 3345, 'Movie': 2620, 'ONA'...","{'Original': 5476, 'Manga': 4550, 'UNKNOWN': 1...","{'24 min. per ep.': 1891, '23 min. per ep.': 1...","{'2008': 33, '2006': 32, '2004': 31, '2003': 3..."


Các cột đều có giá trị không có gì bất thường. Riêng cột Title có số giá trị ít hơn số hàng của dữ liệu (16943 hàng). Ta sẽ in thử distribution ở cột Title để xem.

In [24]:
single_cate_profile.loc['distribution','Title']

{'Shen Lan Qi Yu Wushuang Zhu: Tianmo Pian': 2,
 'Match Shoujo PV': 1,
 'Bouken Shounen Shadar': 1,
 'Fire Emblem Heroes: Fjorm ga Utau - "Koori no Hana"': 1,
 'Dragon Ball Z Movie 04: Super Saiyajin da Son Gokuu': 1,
 '30-pun de Wakaru! Kore made no Love Live!': 1,
 'Crayon Shin-chan Movie 07: Bakuhatsu! Onsen Wakuwaku Daikessen': 1,
 'Nodame Cantabile': 1,
 'Exper Zenon': 1,
 'Kaibutsu-kun: Kaibutsu Land e no Shoutai': 1,
 'Boku no Kanojo ga Majimesugiru Sho-bitch na Ken': 1,
 'Yasashii Fue, Tori, Ishi': 1,
 'Wo Jia Da Shixiong Shi Ge Fanpai': 1,
 'Dumbbell Nan Kilo Moteru?': 1,
 'Attack No.1: Namida no Fushichou': 1,
 'Wushen Zhuzai': 1,
 'Nekopara OVA': 1,
 "Chi's Sweet Home": 1,
 'Kouryuu Densetsu Villgust PV': 1,
 'Märchen Mädchen Specials': 1,
 'Aftermath': 1,
 'Shounen Sunday CM: Ganba! Fly High': 1,
 'Donbei x Kemurikusa': 1,
 'Kimi dake no Boku de Iru kara': 1,
 'Houkago no Pleiades': 1,
 'Ginga Shippuu Sasuraiger': 1,
 'Otogi': 1,
 "Holy Angel's Carol": 1,
 'Kaze wo Mita Sho

Ta thấy rằng bộ dữ liệu có 2 bộ phim trùng tên là: Shen Lan Qi Yu Wushuang Zhu: Tianmo Pian. 

In [25]:
duplicate_titles = anime_df[anime_df['Title'] == "Shen Lan Qi Yu Wushuang Zhu: Tianmo Pian"]
duplicate_titles

,ID,Title,Episodes,Type,Aired,Producers,Studios,Source,Genres,Duration,Score,Scored By,Rank,Popularity
22752,53367.0,Shen Lan Qi Yu Wushuang Zhu: Tianmo Pian,15.0,ONA,"Jan 14, 2023 to Apr 15, 2023","iQIYI, High Energy Studio, Xuanshi Tangmen",CG Year,Web novel,"Action, Fantasy",20 min. per ep.,7.60,385.0,15502.0,14176.0
24119,55740.0,Shen Lan Qi Yu Wushuang Zhu: Tianmo Pian,15.0,ONA,"Jul 15, 2023 to Oct 14, 2023","iQIYI, High Energy Studio",CG Year,Web novel,"Action, Fantasy",22 min. per ep.,7.46,262.0,20462.0,14602.0


Khi truy cập lại thông tin trên nền tảng MyAnimeList, ta thấy 2 bộ phim bị trùng tên có thể phân biệt dựa vào tên Tiếng Anh của chúng (English name). Do đó để tránh nhầm lẫn thông tin ta sẽ đổi giá trị cột Title của 2 bộ phim thành tên Tiếng Anh (dựa vào cột Aired).

In [26]:
# Sắp xếp các hàng trùng lặp theo thời gian 'Aired' (thời gian sớm hơn sẽ đứng trước)
duplicate_titles_sorted = duplicate_titles.sort_values(by='Aired')

# Cập nhật tên Title theo thứ tự đã sắp xếp
anime_df.loc[duplicate_titles_sorted.index[0], 'Title'] = "The Land of Miracles 2nd Season"
anime_df.loc[duplicate_titles_sorted.index[1], 'Title'] = "The Land of Miracles 3rd Season"

# Kiểm tra kết quả
print(anime_df[anime_df['Title'].str.contains("The Land of Miracles")])

            ID                            Title  Episodes Type  \
22752  53367.0  The Land of Miracles 2nd Season      15.0  ONA   
24119  55740.0  The Land of Miracles 3rd Season      15.0  ONA   

                              Aired  \
22752  Jan 14, 2023 to Apr 15, 2023   
24119  Jul 15, 2023 to Oct 14, 2023   

                                        Producers  Studios     Source  \
22752  iQIYI, High Energy Studio, Xuanshi Tangmen  CG Year  Web novel   
24119                   iQIYI, High Energy Studio  CG Year  Web novel   

                Genres         Duration  Score  Scored By     Rank  Popularity  
22752  Action, Fantasy  20 min. per ep.   7.60      385.0  15502.0     14176.0  
24119  Action, Fantasy  22 min. per ep.   7.46      262.0  20462.0     14602.0  


#### Tiếp theo ta tính toán số lượng các cột có nhiều giá trị

Tương tự với cột đơn ta ta tiếp tục lưu tên các cột có nhiều giá trị vào `multi_columns`

In [27]:
multi_columns = ['Genres', 'Producers', 'Studios']

Sau đó tính giá trị thiếu của các cột này

In [28]:
for col in multi_columns:
    percentage_missing = (anime_df[col] == 'UNKNOWN').mean() * 100
    percentage_missing_dic[col] = percentage_missing
    print(f'Phần trăm giá trị còn thiếu của cột \'{col}\' là {percentage_missing:.2f}%')

Phần trăm giá trị còn thiếu của cột 'Genres' là 47.17%
Phần trăm giá trị còn thiếu của cột 'Producers' là 39.26%
Phần trăm giá trị còn thiếu của cột 'Studios' là 22.65%


Rõ ràng, ta nhận thấy rằng các giá trị của cột Producers và Studios có khá nhiều giá trị `UNKNOWN` có thể là do việc tìm kiếm thông tin của Producers hay Studios có thể bị bảo mật hoặc các anime không do các Producers và Studios chuyên nghiệp tổ chức.

Lưu số lượng giá trị khác nhau và đếm số lượng từng loại và lưu vào mảng `multiple_cate_profile`

In [29]:
multi_columns = ['Genres', 'Producers', 'Studios']

# Define function to split and melt columns
def split_and_melt(column):
    df_split = column.str.split(', ', expand=True).melt().drop('variable', axis=1)
    return df_split['value']

# Apply the function to create a long DataFrame
long_df = anime_df[multi_columns].agg(split_and_melt)

# Calculate the distribution and number of unique values
distribution = long_df.agg(lambda column: pd.Series(column).value_counts().to_dict())
num_diff_vals = long_df.nunique() - 1

# Create an empty DataFrame with the correct structure
multiple_cate_profile = pd.DataFrame(index=["num_diff_vals", "distribution"], columns=multi_columns)

# Assign the calculated values to the DataFrame
multiple_cate_profile.loc["num_diff_vals"] = num_diff_vals
multiple_cate_profile.loc["distribution"] = distribution

# Display the resulting DataFrame
multiple_cate_profile


,Genres,Producers,Studios
num_diff_vals,22,1514,1023
distribution,"{'UNKNOWN': 7992, 'Action': 3836, 'Comedy': 37...","{'UNKNOWN': 6651, 'TV Tokyo': 589, 'Aniplex': ...","{'UNKNOWN': 3838, 'Toei Animation': 745, 'Sunr..."


Ta có thể trích xuất cột thể loại để xem thử:

In [30]:
multiple_cate_profile.loc['distribution','Genres']

{'UNKNOWN': 7992,
 'Action': 3836,
 'Comedy': 3794,
 'Fantasy': 3006,
 'Adventure': 2665,
 'Sci-Fi': 2374,
 'Drama': 1954,
 'Romance': 1626,
 'Supernatural': 1093,
 'Mystery': 751,
 'Ecchi': 732,
 'Slice of Life': 625,
 'Horror': 425,
 'Suspense': 344,
 'Hentai': 285,
 'Sports': 271,
 'Award Winning': 210,
 'Avant Garde': 154,
 'Boys Love': 140,
 'Gourmet': 112,
 'Girls Love': 93,
 'Erotica': 52}

Có tổng cộng 21 thể loại, 1513 producer, và 1022 Studios khác nhau còn các giá trị còn lại `UNKNOWN`.


#### Nhận xét chung

- Các giá trị của các cột và các hàng sau khi tiền xử lý không có gì bất thường, các con số đều phù hợp với ý nghĩa của từng cột dữ liệu.
- Phần lớn các giá trị thiếu nằm ở ở các cột chứa thông tin của anime, điều này không quá bất ngờ vì khó mà có thể thu thập đủ dữ liệu của các anime mới ra hoặc các anime không nổi tiếng không được nhiều người biết tới.
- Các cột Episodes, Type và Popularity có giá trị bị thiếu rất ít hoặc không có do đó các cột này rất có giá trị trong việc đánh giá và xử lý trong bộ dữ liệu này.
- Các giá trị số các cột Score, Scored By, Rank là những thông tin hữu ích để đánh giá, xếp hạng anime. Tuy nhiên các cột này có tỉ lệ bị thiếu dữ liệu tương đối cao và nhóm đã quyết định loại bỏ những hàng có giá trị thiếu.
- Các cột số cũng có các khoảng giá trị phù hợp với tên cột.
- Các cột có nhiều giá trị trong một ô là có số lượng giá trị bị mất nhiều nhất còn các cột phân loại có một giá trị thì hầu như dữ liệu rất đầy đủ. Cho thấy là thông tin Producers hay Studios của một anime có thể không được công bố hoặc rất khó để tìm kiếm thông tin này.

#### Thực hiện One-Hot Encoding cho các cột có nhiều giá trị

In [31]:
# Thực hiện One-Hot Encoding
anime_df_encoded = anime_df.copy()

# Chuyển đổi các cột thành dạng One-Hot Encoding
anime_df_encoded = pd.get_dummies(anime_df_encoded, columns=['Genres', 'Producers', 'Studios'], prefix=['Genre', 'Producer', 'Studio'])

# Kiểm tra kết quả
anime_df_encoded.head()

,ID,Title,Episodes,Type,Aired,Source,Duration,Score,Scored By,Rank,...,"Studio_lxtl, LIDENFILMS",Studio_monofilmo,Studio_pH Studio,"Studio_pH Studio, Noovo",Studio_production doA,"Studio_soket, maroyaka",Studio_studio MOTHER,Studio_ufotable,"Studio_ufotable, Shaft, A-1 Pictures, SILVER LINK., Lerche, Wit Studio, Lay-duce, CloverWorks, Drive, Ijigen Tokyo","Studio_ufotable, feel., Studio Flag"
0,1.0,Cowboy Bebop,26.0,TV,"Apr 3, 1998 to Apr 24, 1999",Original,24 min. per ep.,8.75,988000.0,482.0,...,0,0,0,0,0,0,0,0,0,0
1,5.0,Cowboy Bebop: Tengoku no Tobira,1.0,Movie,"Sep 1, 2001",Original,1 hr. 55 min.,8.38,219779.0,2092.0,...,0,0,0,0,0,0,0,0,0,0
2,6.0,Trigun,26.0,TV,"Apr 1, 1998 to Sep 30, 1998",Manga,24 min. per ep.,8.22,380049.0,3602.0,...,0,0,0,0,0,0,0,0,0,0
3,7.0,Witch Hunter Robin,26.0,TV,"Jul 3, 2002 to Dec 25, 2002",Original,25 min. per ep.,7.24,44600.0,30982.0,...,0,0,0,0,0,0,0,0,0,0
4,8.0,Bouken Ou Beet,52.0,TV,"Sep 30, 2004 to Sep 29, 2005",Manga,23 min. per ep.,6.93,6891.0,45862.0,...,0,0,0,0,0,0,0,0,0,0


Vậy cuối cùng ta sẽ xuất file vừa tiền xử lý để dùng cho việc trả lời các câu hỏi.

In [32]:
anime_df_encoded.to_csv('D:/Intro2DS_FinalProject/data_exploration/anime_preprocessing.csv', index=False)